### Import required libraries and load dataset

In [2]:
from sklearn import datasets,svm

In [4]:
iris = datasets.load_iris()

In [5]:
import pandas as pd
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x:iris.target_names[x])

In [6]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [7]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [8]:
df["flower"].value_counts()

setosa        50
versicolor    50
virginica     50
Name: flower, dtype: int64

### Approach 1 : Use train test split and manually tune parameters by try and error

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,test_size=0.3)

In [15]:
model = svm.SVC(kernel="rbf",C=30,gamma="auto")
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9333333333333333

In [23]:
model = svm.SVC(kernel="linear",C=25,gamma="auto")
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9555555555555556

### Approach 2 : Use K fold cross validation

In [24]:
from sklearn.model_selection import cross_val_score

In [27]:
cross_val_score(svm.SVC(kernel="rbf",C=30,gamma="auto"),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.9       , 0.93333333, 1.        ])

In [28]:
cross_val_score(svm.SVC(kernel="linear",C=30,gamma="auto"),iris.data,iris.target,cv=5)

array([1. , 1. , 0.9, 0.9, 1. ])

Above approach is very manual, So we use for loop to automate this

In [37]:
import numpy as np

In [40]:
kernels=["rbf","linear","poly"]
C=[1,10,20]
avg_score={}
for kval in kernels:
    for cval in C:
        cv_score = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma="auto"),iris.data,iris.target,cv=5)
        avg_score[kval + "_"+str(cval)]=np.average(cv_score)
avg_score

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666,
 'poly_1': 0.9666666666666666,
 'poly_10': 0.9666666666666666,
 'poly_20': 0.9533333333333334}

### Use GridSearch CV

In [45]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(svm.SVC(gamma="auto"),{
    "C":[1,10,20],
    "kernel":["rbf","linear"]
},cv=5,return_train_score=False)

In [46]:
clf.fit(iris.data,iris.target)

GridSearchCV(cv=5, estimator=SVC(gamma='auto'),
             param_grid={'C': [1, 10, 20], 'kernel': ['rbf', 'linear']})

In [47]:
df=pd.DataFrame(clf.cv_results_)

In [48]:
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000639,0.000233,0.000216,0.000028,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000427,0.000061,0.000204,0.000017,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000491,0.000034,0.000194,0.000022,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000481,0.000209,0.000183,0.000033,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000449,0.000007,0.000226,0.000054,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000407,0.000054,0.000165,0.000014,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [49]:
df[["param_C","param_kernel","mean_test_score"]]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [50]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [51]:
clf.best_score_

0.9800000000000001